<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-22 10:22:16
-------------------
qualified stocks: 93
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.39 L
Current:  1.37 C
-------------------
Today PnL: -65.73 K (-0.48%)
Current PnL: -17.98 L (-12.23%)
CY Booked + Current PnL: -9.25 L (-6.29%)
-------------------
Total profit:  3.55 L
Total loss:  -21.53 L
-------------------
Total Booked + Current PnL: 17.73 L (14.77%)
Total Booked PnL: 35.71 L (29.75%)
Curr Year Booked PnL: 8.73 L (6.36%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 77.09 L (56.13%)
Deployed:  1.20 C
Current:  1.37 C
CAGR/XIRR %: 8.58%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-18.25,H-LC,7.15,112728.0,8416.0,8229.0,-0.39,8.07,7.30,15.96,9.0,1.02,0.84,26.33,X5K,ATH,METALS
4,APOLLOHOSP,7098.95,8285.00,-12.98,H-LC,7.64,173998.0,17821.0,8265.0,-0.22,11.41,4.75,16.71,22.0,2.16,1.30,30.84,X40N,BTT,HEALTHCARE
37,HINDUNILVR,2413.81,2723.00,-6.25,H-LC,7.33,294683.0,24336.0,10284.0,-0.64,9.00,3.49,12.81,24.0,2.37,2.20,23.24,XY25,NTT,FMCG
68,SBIN,760.30,863.00,-12.30,M-LC,3.93,217936.0,16456.0,10766.0,-0.40,8.17,4.94,13.51,60.0,1.53,1.63,21.79,XY25,NTT,BANKS
55,MEDANTA,1087.93,1486.00,12.31,H-MC,14.47,160393.0,34193.0,11981.0,-0.46,27.09,7.47,36.59,108.0,2.85,1.20,43.06,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-30.43,H-MC,7.32,179208.0,-600.0,113009.0,0.49,-0.33,63.06,62.52,88.0,-0.01,1.34,33.64,XR,NTT,BANKS
58,PGHH,13388.00,18062.58,-30.41,H-MC,5.58,201525.0,705.0,69405.0,0.25,0.35,34.44,34.92,80.0,0.01,1.50,5.04,X40,ATH,FMCG
56,NATIONALUM,189.63,247.44,-49.46,H-MC,1.85,100806.0,302.0,30333.0,-0.66,0.30,30.09,30.49,79.0,0.01,0.75,32.80,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-39.19,H-LC,3.05,145782.0,-4150.0,18748.0,-1.25,-2.77,12.86,9.74,4.0,-0.22,1.09,3.30,X40,NTT,FMCG
53,LTIM,5564.16,7230.2,-5.20,H-LC,3.63,187830.0,-12480.0,72465.0,-0.33,-6.23,38.58,29.94,16.0,-0.17,1.40,29.76,X200,ATH,IT
57,NESTLEIND,1134.30,1377.0,-11.28,H-LC,3.67,273172.0,7746.0,49034.0,-1.02,2.92,17.95,21.40,11.0,0.16,2.04,10.93,XY25,NTT,FMCG
21,CIPLA,1492.70,1795.0,-18.41,H-LC,3.95,218036.0,13536.0,27887.0,-0.08,6.62,12.79,20.25,10.0,0.49,1.63,15.93,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-11.75,H-LC,5.70,172074.0,-29312.0,116924.0,-1.41,-14.56,67.95,43.51,27.0,-0.25,1.28,19.78,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,SURYODAY,189.78,240.0,44.74,H-SC,5.49,102416.0,-47510.0,87187.0,1.06,-31.69,85.13,26.46,135.0,-0.54,0.76,31.20,XR,NTT,BANKS
73,SONACOMS,578.05,1006.0,-37.22,M-SC,14.18,78199.0,-22960.0,97850.0,0.95,-22.70,125.13,74.03,202.0,-0.23,0.58,10.27,AR,BTT,AUTO
74,STARHEALTH,551.56,761.0,8.53,H-SC,1.86,161190.0,-39026.0,115057.0,0.87,-19.49,71.38,37.97,144.0,-0.34,1.20,29.42,XY24,NTT,INSURANCE
72,SIS,477.00,528.0,2197.41,H-SC,5.63,61812.0,-19278.0,27945.0,0.78,-23.77,45.21,10.69,156.0,-0.69,0.46,25.27,OX40N,NTT,MISC
85,UNIONBANK,123.87,163.0,-13.18,M-LC,12.69,156349.0,15509.0,28987.0,0.72,11.01,18.54,31.59,66.0,0.54,1.17,40.42,XY24,NTT,BANKS


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,RAJOOENG,102.50,143.33,-31.26,H-SC,9.39,104170.0,1670.0,39158.0,-3.54,1.63,37.59,39.83,114.0,0.04,0.78,3.14,AR,ATH,MISC
29,EASEMYTRIP,18.12,26.40,-8.58,M-SC,27.17,88115.0,-88700.0,169498.0,-3.11,-50.17,192.36,45.70,197.0,-0.52,0.66,2.50,XY24,NTT,TRAVEL
23,COFFEEDAY,59.14,80.00,-41.65,L-SC,24.67,86227.0,-27322.0,67369.0,-2.75,-24.06,78.13,35.27,268.0,-0.41,0.64,111.04,XR,NTT,HOTELS
35,HEROMOTOCO,4311.81,5949.07,-5.85,H-MC,12.17,174636.0,23723.0,33583.0,-2.11,15.72,19.23,37.97,93.0,0.71,1.30,45.00,AR,ATH,AUTO
52,LAOPALA,369.40,464.00,113.76,H-SC,1.92,67991.0,-32855.0,58683.0,-1.74,-32.58,86.31,25.61,142.0,-0.56,0.51,26.06,AR,NTT,CERAMICS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,99.15,M-SC,9.11,85838.0,-14939.0,15030.0,-0.43,-14.82,17.51,0.09,245.0,-0.99,0.64,12.77,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,86.70,H-SC,15.99,117828.0,-24651.0,44174.0,-0.41,-17.30,37.49,13.70,163.0,-0.56,0.88,39.07,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-17.90,H-SC,3.12,117090.0,-26713.0,53463.0,0.42,-18.58,45.66,18.60,149.0,-0.50,0.87,29.21,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,112.05,M-SC,2.00,138992.0,-35567.0,35596.0,-1.29,-20.38,25.61,0.02,221.0,-1.00,1.04,20.90,OX40N,NTT,PAINTS
50,KANSAINER,299.63,340.0,-69.25,H-SC,3.25,212472.0,-57195.0,93530.0,-0.86,-21.21,44.02,13.47,138.0,-0.61,1.59,7.47,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-27.72,H-LC,16.19,239109.0,-56825.0,150734.0,-1.20,-19.20,63.04,31.73,1.0,-0.38,1.79,2.08,X40,BTT,IT
45,INFY,1461.46,2275.00,-18.52,H-LC,12.08,266585.0,4984.0,140650.0,-0.47,1.91,52.76,55.67,3.0,0.04,1.99,8.13,X40,BTT,IT
47,ITC,411.90,452.00,-39.19,H-LC,3.05,145782.0,-4150.0,18748.0,-1.25,-2.77,12.86,9.74,4.0,-0.22,1.09,3.30,X40,NTT,FMCG
89,VBL,492.64,671.64,-8.45,H-LC,9.13,277985.0,12945.0,83368.0,-0.11,4.88,29.99,36.33,5.0,0.16,2.08,18.83,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.13,H-LC,12.22,199972.0,-16547.0,109445.0,-0.27,-7.64,54.73,42.91,7.0,-0.15,1.49,4.73,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,12.31,H-MC,14.47,160393.0,34193.0,11981.0,-0.46,27.09,7.47,36.59,108.0,2.85,1.20,43.06,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-8.50,H-LC,9.44,228585.0,44104.0,26264.0,-0.57,23.91,11.49,38.14,38.0,1.68,1.71,37.05,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,12.31,H-MC,14.47,160393.0,34193.0,11981.0,-0.46,27.09,7.47,36.59,108.0,2.85,1.20,43.06,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,10.55,H-MC,5.33,190830.0,26160.0,41811.0,0.37,15.89,21.91,41.28,89.0,0.63,1.42,25.14,X40N,ATH,AC
51,KPIGREEN,497.21,731.35,9.79,H-SC,10.97,130914.0,5617.0,53387.0,0.44,4.48,40.78,47.09,141.0,0.11,0.98,63.85,MH,ATH,POWER
90,VOLTAS,1278.28,1918.49,-2.48,H-MC,7.22,204750.0,13008.0,83026.0,0.17,6.78,40.55,50.08,99.0,0.16,1.53,14.04,XY25,ATH,AC
19,CAMS,3643.00,5211.76,-3.83,H-SC,5.72,109368.0,7364.0,36562.0,0.27,7.22,33.43,43.06,122.0,0.20,0.82,25.57,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-5.85,H-MC,12.17,174636.0,23723.0,33583.0,-2.11,15.72,19.23,37.97,93.0,0.71,1.30,45.00,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-49.41,H-SC,12.19,141468.0,18132.0,115636.0,0.35,14.70,81.74,108.45,119.0,0.16,1.06,40.34,AR,ATH,MISC
51,KPIGREEN,497.21,731.35,9.79,H-SC,10.97,130914.0,5617.0,53387.0,0.44,4.48,40.78,47.09,141.0,0.11,0.98,63.85,MH,ATH,POWER
91,WHIRLPOOL,1219.98,2270.00,-42.43,M-SC,5.39,97320.0,5822.0,72932.0,-1.41,6.36,74.94,86.07,223.0,0.08,0.73,40.78,XR,NTT,DURABLES
61,RAJOOENG,102.50,143.33,-31.26,H-SC,9.39,104170.0,1670.0,39158.0,-3.54,1.63,37.59,39.83,114.0,0.04,0.78,3.14,AR,ATH,MISC


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,75.41,137.00,7044.44,L-SC,7.04,78384.0,-15426.0,92046.0,-0.87,-16.44,117.43,81.67,269.0,-0.17,0.59,51.79,XR,NTT,CERAMICS
23,COFFEEDAY,59.14,80.00,-41.65,L-SC,24.67,86227.0,-27322.0,67369.0,-2.75,-24.06,78.13,35.27,268.0,-0.41,0.64,111.04,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-1.31,H-SC,15.88,91826.0,-9137.0,103662.0,-1.14,-9.05,112.89,93.63,148.0,-0.09,0.69,36.77,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-17.80,H-SC,12.48,93435.0,-4545.0,26414.0,-0.27,-4.64,28.27,22.32,152.0,-0.17,0.70,35.51,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-10.13,H-LC,6.80,96540.0,-1205.0,38606.0,-0.49,-1.23,39.99,38.26,35.0,-0.03,0.72,26.41,X40,BTT,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-41.65,L-SC,24.67,86227.0,-27322.0,67369.0,-2.75,-24.06,78.13,35.27,268.0,-0.41,0.64,111.04,XR,NTT,HOTELS
35,HEROMOTOCO,4311.81,5949.07,-5.85,H-MC,12.17,174636.0,23723.0,33583.0,-2.11,15.72,19.23,37.97,93.0,0.71,1.30,45.00,AR,ATH,AUTO
28,DMART,3764.92,5201.00,-8.50,H-LC,9.44,228585.0,44104.0,26264.0,-0.57,23.91,11.49,38.14,38.0,1.68,1.71,37.05,X40N,NTT,FMCG
4,APOLLOHOSP,7098.95,8285.00,-12.98,H-LC,7.64,173998.0,17821.0,8265.0,-0.22,11.41,4.75,16.71,22.0,2.16,1.30,30.84,X40N,BTT,HEALTHCARE
21,CIPLA,1492.70,1795.00,-18.41,H-LC,3.95,218036.0,13536.0,27887.0,-0.08,6.62,12.79,20.25,10.0,0.49,1.63,15.93,X40N,BTT,PHARMA


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.79
1,25,41.43
2,50,71.05


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.55
LC    36.23
MC    26.30
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.46
X40      14.34
X40N     13.96
XY25     11.84
XR        9.11
AR        8.45
OX40N     7.43
X5K       2.23
MH        1.73
X200      1.40
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.00
H-MC    23.40
H-SC    22.75
M-SC    13.20
M-LC     7.19
M-MC     2.57
L-SC     1.60
L-LC     1.04
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.57,0.32,30.21
IT,11.87,-17.43,76.08
FINANCE,8.09,-20.36,64.89
BANKS,7.62,-11.82,59.30
MISC,5.86,-17.26,84.48
PAINTS,5.52,-18.15,41.44
ELECTRICAL,4.75,-10.18,57.90
AUTO,4.52,-11.49,60.00
AC,3.68,1.78,31.48


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2828039.0,23
AR,1077842.0,10
XR,1075822.0,13
X40,815187.0,11
X40N,496723.0,11
OX40N,463829.0,10
XY25,462183.0,8
SR,193296.0,2
X5K,140192.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2732962.0,27
M-SC,1481325.0,17
H-MC,1240837.0,18
H-LC,1151828.0,19
M-LC,417639.0,5
M-MC,333551.0,2
L-SC,247232.0,3
L-MC,58690.0,1
L-LC,45234.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,927002.0,6
M-SC,XY24,869793.0,7
H-SC,AR,656750.0,5
H-LC,X40,523523.0,6
H-SC,XR,517033.0,6
H-MC,XY24,485195.0,5
M-MC,XY24,333551.0,2
H-LC,AR,228218.0,2
H-MC,X40,209578.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 58.0 seconds
